In [1]:
import numpy as np

In [2]:
#naive bayes fit function
def fit(X_train,Y_train):
    result = {}
    classes = set(Y_train)
    for current_class in classes:
        result[current_class] = {}
        result["total_data"] = len(Y_train)
        current_class_rows = (Y_train == current_class)
        X_train_rows = X_train[current_class_rows]
        Y_train_rows = Y_train[current_class_rows]
        features = X_train.shape[1]
        result[current_class]["total_count"] = len(Y_train_rows)
        for j in range(1,features+1):
            result[current_class][j] = {}
            all_values = set(X_train[:,j-1])
            for current_value in all_values:
                result[current_class][j][current_value] = (X_train_rows[:,j-1] == current_value).sum()
    return result                

In [3]:
# probability function
def prob(dictionary,x,current_class):
    outcome = np.log(dictionary[current_class]["total_count"])-np.log(dictionary["total_data"])
    all_possible_feauture_values = len(dictionary[current_class].keys()) - 1
    for j in range(1,all_possible_feauture_values+1):
        xj = x[j-1]
        count_for_xj_in_currentclass = (dictionary[current_class][j][xj]) +1
        count_for_total_classvalues = dictionary[current_class]["total_count"] +len(dictionary[current_class][j].keys())
        prob_dynamic = np.log(count_for_xj_in_currentclass)-np.log(count_for_total_classvalues)
        outcome = outcome + prob_dynamic
        
    return outcome


In [4]:
# single point prediction function
def SinglePointPrediction(dictionary,x):
    firstrun = True
    best_prob = -1
    best_class = -10
    classes = dictionary.keys()
    for current_class in classes:
        if current_class == "total_data":
            continue
        if firstrun or prob(dictionary,x,current_class) > best_prob:
            best_class = current_class
            best_prob = prob(dictionary,x,current_class)
        firstrun = False
        
    return best_class

In [20]:
#predict function
def predict(dictionary,X_test):
    y_pred = []
    for x in X_test:
        x_class = SinglePointPrediction(dictionary,x)
        y_pred.append(x_class)
    return y_pred

In [6]:
#labelling the continous data
def makeLabelled(column):
    second_limit = column.mean()
    first_limit = 0.5 * second_limit
    third_limit = 1.5 * second_limit
    
    for i in range(0,len(column)):
        if(column[i] < first_limit):
            column[i] = 0
        elif(column[i] < second_limit):
            column[i] = 1
        elif(column[i] < third_limit):
            column[i] = 2
        else:
            column[i] = 3
            
    return column



In [7]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
Y = iris.target

In [8]:
for i in range(0,X.shape[-1]):
    X[:,i] = makeLabelled(X[:,i])

In [15]:
from sklearn import model_selection
X_train,X_test,Y_train,Y_test = model_selection.train_test_split(X,Y,test_size = 0.25,random_state = 0)

In [10]:
dictionary = fit(X_train,Y_train)

In [23]:
Y_pred = predict(dictionary,X_test)

In [24]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(Y_test,Y_pred))
print(confusion_matrix(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       0.94      1.00      0.97        16
           2       1.00      0.89      0.94         9

   micro avg       0.97      0.97      0.97        38
   macro avg       0.98      0.96      0.97        38
weighted avg       0.98      0.97      0.97        38

[[13  0  0]
 [ 0 16  0]
 [ 0  1  8]]
